In [ ]:
import zipfile
import pandas as pd
import torch
from huggingface_hub import hf_hub_download
import numpy as np
from tqdm.auto import tqdm
import time
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [ ]:
zip_path = "/content/drive/MyDrive/finaldata.csv.zip"
extract_dir = "/content"
target_file = "finaldata.csv"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Check if the file exists inside the archive
    if target_file in zip_ref.namelist():
        # Extract only the target file
        zip_ref.extract(target_file, extract_dir)
        print(f"✅ Extracted {target_file} to {extract_dir}")
    else:
        print(f"⚠️ {target_file} not found in the zip file.")

✅ Extracted finaldata.csv to /content


In [ ]:
data = pd.read_csv("finaldata.csv")

/tmp/ipython-input-1908380124.py:1: DtypeWarning: Columns (76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("finaldata.csv")


In [ ]:
data['mentor_expectation'] = data.apply(
        lambda row: row['person1_expectations']
        if row['person1_is_mentor'] == True
        else row['person2_expectations'],
        axis=1
    )

In [ ]:
freq = data['mentor_expectation'].value_counts().reset_index()
freq = freq[:200]

In [ ]:
device_to_use = 0 if torch.cuda.is_available() else -1
print(f"✅ Using GPU: {torch.cuda.get_device_name(0)}" if device_to_use == 0 else "⚠️ Using CPU (slower)")

✅ Using GPU: Tesla T4


In [ ]:
freq['mentor_expectation'] = freq['mentor_expectation'].str.replace(r"[\n\r]+", " ", regex=True)
freq['mentor_expectation'] = freq['mentor_expectation'].str.replace(r"[^\w\s]", " ", regex=True)
freq['mentor_expectation'] = freq['mentor_expectation'].str.replace(r"\s+", " ", regex=True)
freq['mentor_expectation'] = freq['mentor_expectation'].str.strip()

In [ ]:
behaviour_en = "I expect mentees to take full ownership of their work, stay consistent and prepared, communicate proactively, and remain curious and self-aware throughout the process."
outcome_en = "What matters most to me is seeing tangible, measurable results and real-world impact that clearly reflect effective execution."
strategy_en = "For mentorship to be effective, I need the mentee’s values, long-term vision, goals, and strategic direction to be aligned with mine."
knowledge_en = "I mentor with the intention of transferring practical knowledge, developing the mentee’s skills, and guiding them through real-world challenges based on my experience."
mutual_en = "I value a mentoring relationship built on mutual trust, open communication, and collaborative problem-solving, where both of us actively work together."

In [ ]:
behaviour_sp = "Valoro una relación de mentoría basada en la confianza mutua, la comunicación abierta y la resolución colaborativa de problemas, donde ambos trabajemos activamente juntos."
outcome_sp = "Lo que más me importa es ver resultados tangibles y medibles, así como un impacto real en el mundo, que reflejen claramente una ejecución eficaz."
strategy_sp = "Para que la mentoría sea efectiva, es necesario que los valores, la visión a largo plazo, los objetivos y la dirección estratégica del aprendiz estén alineados con los míos."
knowledge_sp = "Mi objetivo al ejercer la mentoría es transmitir conocimientos prácticos, desarrollar las habilidades del aprendiz y guiarlo a través de los desafíos del mundo real basándome en mi experiencia."
mutual_sp = "Valoro una relación de mentoría basada en la confianza mutua, la comunicación abierta y la resolución colaborativa de problemas, donde ambos trabajemos activamente juntos."

In [ ]:
behaviour_ar = "أتوقع من المتدربين أن يتحملوا المسؤولية الكاملة عن عملهم، وأن يكونوا ملتزمين ومنظمين، وأن يتواصلوا بشكل استباقي، وأن يظلوا فضوليين وواعين بذاتهم طوال العملية."
outcome_ar = "أكثر ما يهمني هو رؤية نتائج ملموسة وقابلة للقياس وتأثير حقيقي على أرض الواقع يعكس بوضوح التنفيذ الفعال."
strategy_ar = "لكي يكون التوجيه فعالاً، يجب أن تتوافق قيم المتدرب ورؤيته طويلة المدى وأهدافه وتوجهاته الاستراتيجية مع قيمي أنا."
knowledge_ar = "أقوم بعملية التوجيه والإرشاد بهدف نقل المعرفة العملية، وتطوير مهارات المتدرب، وتوجيهه خلال التحديات الواقعية بناءً على خبرتي."
mutual_ar = "أنا أقدر علاقة التوجيه المبنية على الثقة المتبادلة والتواصل المفتوح وحل المشكلات بشكل تعاوني، حيث نعمل معًا بنشاط."

In [ ]:
behaviour_id = "Saya mengharapkan para peserta program bimbingan untuk bertanggung jawab penuh atas pekerjaan mereka, tetap konsisten dan siap, berkomunikasi secara proaktif, serta tetap memiliki rasa ingin tahu dan kesadaran diri sepanjang proses berlangsung."
outcome_id = "Hal yang paling penting bagi saya adalah melihat hasil yang nyata dan terukur serta dampak di dunia nyata yang jelas mencerminkan pelaksanaan yang efektif."
strategy_id = "Agar program bimbingan berjalan efektif, nilai-nilai, visi jangka panjang, tujuan, dan arah strategis dari orang yang dibimbing perlu selaras dengan nilai-nilai saya."
knowledge_id = "Saya melakukan pendampingan dengan tujuan untuk mentransfer pengetahuan praktis, mengembangkan keterampilan peserta bimbingan, dan membimbing mereka menghadapi tantangan di dunia nyata berdasarkan pengalaman saya."
mutual_id = "Saya menghargai hubungan bimbingan yang dibangun di atas kepercayaan timbal balik, komunikasi terbuka, dan penyelesaian masalah secara kolaboratif, di mana kita berdua aktif bekerja sama."

In [ ]:
behaviour_ru = "Я ожидаю, что участники программы наставничества будут полностью отвечать за свою работу, проявлять последовательность и подготовленность, активно общаться, а также сохранять любознательность и самосознание на протяжении всего процесса."
outcome_ru = "Для меня самое важное — это видеть ощутимые, измеримые результаты и реальное влияние на окружающий мир, которые явно свидетельствуют об эффективном выполнении поставленных задач."
strategy_ru = "Для того чтобы наставничество было эффективным, необходимо, чтобы ценности, долгосрочное видение, цели и стратегическое направление подопечного совпадали с моими."
knowledge_ru = "В рамках наставничества я ставлю перед собой цель передать практические знания, развить навыки подопечного и помочь ему справиться с реальными задачами, опираясь на собственный опыт."
mutual_ru = "Я ценю наставнические отношения, построенные на взаимном доверии, открытом общении и совместном решении проблем, где мы оба активно работаем вместе."

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/bge-m3-zeroshot-v2.0")
model = AutoModel.from_pretrained("MoritzLaurer/bge-m3-zeroshot-v2.0")

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at MoritzLaurer/bge-m3-zeroshot-v2.0 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def embed_sentences(sentences):
    inputs = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**inputs)

    # CLS token pooling
    embeddings = outputs.last_hidden_state[:, 0]

    # Normalize (important for cosine similarity)
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings

In [ ]:
behaviour_sentences = [behaviour_en,behaviour_sp,behaviour_ar,behaviour_id,behaviour_ru]
outcome_sentences = [outcome_en,outcome_sp,outcome_ar,outcome_id,outcome_ru]
strategy_sentences = [strategy_en,strategy_sp,strategy_ar,strategy_id,strategy_ru]
knowledge_sentences = [knowledge_en,knowledge_sp,knowledge_ar,knowledge_id,knowledge_ru]
mutual_sentences = [mutual_en,mutual_sp,mutual_ar,mutual_id,mutual_ru]

behaviour_embeddings = embed_sentences(behaviour_sentences)
outcome_embeddings = embed_sentences(outcome_sentences)
strategy_embeddings = embed_sentences(strategy_sentences)
knowledge_embeddings = embed_sentences(knowledge_sentences)
mutual_embeddings = embed_sentences(mutual_sentences)

# Average to get cluster anchor
behaviour_anchor = behaviour_embeddings.mean(dim=0)
behaviour_anchor = F.normalize(behaviour_anchor, p=2, dim=0)
outcome_anchor = outcome_embeddings.mean(dim=0)
outcome_anchor = F.normalize(outcome_anchor, p=2, dim=0)
strategy_anchor = strategy_embeddings.mean(dim=0)
strategy_anchor = F.normalize(strategy_anchor, p=2, dim=0)
knowledge_anchor = knowledge_embeddings.mean(dim=0)
knowledge_anchor = F.normalize(knowledge_anchor, p=2, dim=0)
mutual_anchor = mutual_embeddings.mean(dim=0)
mutual_anchor = F.normalize(mutual_anchor, p=2, dim=0)


In [ ]:
behaviour_score_list = []
outcome_score_list = []
strategy_score_list = []
knowledge_score_list = []
mutual_score_list = []
for mentor_text in tqdm(freq['mentor_expectation']):
  mentor_embedding = embed_sentences([mentor_text])[0]
  behaviour_score = torch.dot(mentor_embedding, behaviour_anchor)
  behaviour_score_list.append(behaviour_score.item())
  outcome_score = torch.dot(mentor_embedding, outcome_anchor)
  outcome_score_list.append(outcome_score.item())
  strategy_score = torch.dot(mentor_embedding, strategy_anchor)
  strategy_score_list.append(strategy_score.item())
  knowledge_score = torch.dot(mentor_embedding, knowledge_anchor)
  knowledge_score_list.append(knowledge_score.item())
  mutual_score = torch.dot(mentor_embedding, mutual_anchor)
  mutual_score_list.append(mutual_score.item())

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
freq['behaviour_confidence_score'] = behaviour_score_list
freq['outcome_confidence_score'] = outcome_score_list
freq['strategy_confidence_score'] = strategy_score_list
freq['knowledge_confidence_score'] = knowledge_score_list
freq['mutual_confidence_score'] = mutual_score_list

In [ ]:
clusters = ["behaviour_confidence_score", "outcome_confidence_score", "strategy_confidence_score", "knowledge_confidence_score", "mutual_confidence_score"]

# Find subject with highest score for each student
freq["highest_score_cluster"] = (
    freq[clusters]
    .idxmax(axis=1)
    .str.replace("_confidence_score", "")
)

In [ ]:
freq

,mentor_expectation,count,behaviour_confidence_score,outcome_confidence_score,strategy_confidence_score,knowledge_confidence_score,mutual_confidence_score,highest_score_cluster
0,I am very excited for people who want to start...,2808,0.752433,0.802535,0.621314,0.777546,0.757219,outcome
1,Be willing to communicate to me values goals o...,2402,0.727357,0.547470,0.824289,0.652290,0.680863,strategy
2,I take my responsibility as a mentor very seri...,837,0.716761,0.793616,0.545444,0.757442,0.724366,outcome
3,La relación le permite al beneficiario a refle...,641,0.861869,0.848712,0.803295,0.872235,0.858046,knowledge
4,Dar el mayor apoyo y compartir los conocimient...,625,0.861920,0.849923,0.812457,0.889425,0.865235,knowledge
...,...,...,...,...,...,...,...,...
195,El compromiso de la persona a la cual voy a ay...,97,0.818150,0.818249,0.719433,0.843582,0.809268,knowledge
196,Mi intención es que juntos logremos que pases ...,97,0.735554,0.782547,0.622026,0.765900,0.729121,outcome
197,To work with entrepreneurs who have the abilit...,97,0.917770,0.907836,0.877068,0.914001,0.928122,mutual
198,Be open and be natural Be honest and clear abo...,97,0.879674,0.852749,0.853535,0.879227,0.868520,behaviour


In [ ]:
freq.to_csv("finaldata_expectations.csv")